# Database Explorer

Inspect tables, recent documents, and sample chunks.

In [1]:
from pathlib import Path
import os
import sys

ROOT = Path.cwd().resolve()
if (ROOT / 'backend').exists():
    PROJECT_ROOT = ROOT
elif ROOT.name == 'notebook':
    PROJECT_ROOT = ROOT.parent
else:
    PROJECT_ROOT = ROOT

BACKEND_DIR = PROJECT_ROOT / 'backend'
if str(BACKEND_DIR) not in sys.path:
    sys.path.insert(0, str(BACKEND_DIR))

os.chdir(BACKEND_DIR)

In [2]:
import pandas as pd
from app.core.database import db_manager

await db_manager.init_postgres()

async with db_manager.pg_pool.acquire() as conn:
    table_info = await conn.fetch('''
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name
    ''')

tables = [r['table_name'] for r in table_info]
pd.DataFrame({'table_name': tables})

C:\Users\balas\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,table_name
0,chunks
1,documents
2,feedback
3,holographic_storage


In [3]:
async with db_manager.pg_pool.acquire() as conn:
    docs = await conn.fetch('''
        SELECT id, filename, created_at
        FROM documents
        ORDER BY created_at DESC
        LIMIT 20
    ''')

pd.DataFrame([dict(r) for r in docs])

,id,filename,created_at
0,45,Large_language_Model.pdf,2026-02-13 14:49:11.069348
1,44,Holographic_Storage.pdf,2026-02-13 14:48:50.267169
2,43,Holographic_data_storage_technology.pdf,2026-02-13 14:47:55.456234
3,42,Graph_databases.pdf,2026-02-13 14:47:27.800211
4,41,A_New_Swarm_Intelligence_Coordination_Model_In...,2026-02-13 14:47:08.481283
5,40,NBala_swamy_resume.pdf,2026-02-06 06:56:39.583519


In [4]:
DOCUMENT_ID = None  # Set an integer id to inspect a specific document's chunks.

query = '''
    SELECT id, document_id, chunk_index, LEFT(content, 300) AS content_preview
    FROM chunks
'''
params = []
if DOCUMENT_ID is not None:
    query += ' WHERE document_id = $1'
    params = [DOCUMENT_ID]
query += ' ORDER BY id DESC LIMIT 25'

async with db_manager.pg_pool.acquire() as conn:
    rows = await conn.fetch(query, *params)

pd.DataFrame([dict(r) for r in rows])

,id,document_id,chunk_index,content_preview
0,566,45,205,"Moreover, for scientific writing, LLMs can hel..."
1,565,45,204,Task Dataset/Benchmark Top-1 Top-2 Top-3 Model...
2,564,45,203,"In addition, LLMs can aid scientists 32 Table ..."
3,563,45,202,They can generate real-time transcriptions for...
4,562,45,201,"For students, by analyzing their learning styl..."
5,561,45,200,"They can also simulate patient interactions, e..."
6,560,45,199,"Moreover, LLMs can also enhance patient intera..."
7,559,45,198,"Moreover, LLMs play a cru- cial role in data a..."
8,558,45,197,This allows them to perform tasks ranging from...
9,557,45,196,"LLMs, which are capable of understanding and g..."


In [5]:
await db_manager.close()